# Inference of Trained Retinanet model 
### Import Libraries

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from numpy import genfromtxt
import time
import json

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
#setup_gpu(gpu)

Using TensorFlow backend.


### Import Data and Model 

In [2]:
file_location = 'list_of_img_in_val_set4.csv'
place_to_store_results = 'Evaluations/'
path_img_folder = '../../../03 Data/Simple Dataset/'

# import data
file = open(file_location)
file_paths = list(file)  

print(file_paths)

# import model
model_path = os.path.join('..', 'snapshots', 'resnet50_csv_25.h5');

model = models.load_model(model_path, backbone_name='resnet50');

# If model is not converted to inference model, use line below: 
model = models.convert_model(model)


labels_to_names = {0: 'Brick'};

#labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

['colorIMG_75.png\n', 'colorIMG_32.png\n', 'colorIMG_114.png\n', 'colorIMG_95.png\n', 'colorIMG_43.png\n', 'colorIMG_67.png\n', 'colorIMG_24.png\n', 'colorIMG_12.png\n', 'colorIMG_159.png\n', 'colorIMG_51.png\n', 'colorIMG_103.png\n', 'colorIMG_96.png\n', 'colorIMG_50.png\n', 'colorIMG_179.png\n', 'colorIMG_94.png\n', 'colorIMG_88.png\n', 'colorIMG_77.png\n', 'colorIMG_108.png\n', 'colorIMG_37.png\n', 'colorIMG_152.png\n', 'colorIMG_9.png\n', 'colorIMG_45.png\n', 'colorIMG_99.png\n', 'colorIMG_111.png\n', 'colorIMG_201.png\n', 'colorIMG_78.png\n', 'colorIMG_47.png\n', 'colorIMG_15.png\n', 'colorIMG_46.png\n', 'colorIMG_183.png\n', 'colorIMG_62.png\n', 'colorIMG_136.png\n', 'colorIMG_116.png\n', 'colorIMG_147.png\n', 'colorIMG_53.png\n', 'colorIMG_107.png\n', 'colorIMG_132.png\n', 'colorIMG_44.png\n', 'colorIMG_190.png\n', 'colorIMG_55.png\n']
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 

In [7]:
def read_annotations_from_json(img_name):
    print(path_img_folder + img_name.strip('.png') + '.json')
    with open(path_img_folder + img_name.strip('.png') + '.json') as json_file:
        data = json.load(json_file)
        shapes_list = data['shapes']
        annotation_list = []
        for annotation in shapes_list: 
            rect = cv2.boundingRect(np.float32(annotation['points']))
            annotation_list.append(rect)
    return annotation_list; 

def filter_bounding_boxes(bounding_boxes, scores):
    list_of_BB = []
    for box, score in zip(bounding_boxes, scores):
        if score < 0.7: 
            break
        list_of_BB.append(box)
    return list_of_BB


### Infer Images

In [8]:
accept_BB_threshold = 0.7

for path in file_paths:
    image = read_image_bgr(path_img_folder + path.strip('\n'))

    # create copy to draw on 
    draw = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    
    # preprocess image 
    # TODOD: check if preprocess_image convert the image to RGB format
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image 
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    
    # correct for image scale
    boxes /= scale
    
    print(len(boxes[0]))
    # print detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < accept_BB_threshold:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    
    # load json file containing annotations
    annotations = read_annotations_from_json(path.strip('\n'))
    
    
    # find overlaping bounding boxes 
    predictions = filter_bounding_boxes(boxes[0], scores[0])
        #calculate intersection and union of two bounding boxes
    
    # calculate true positives, false negatives, false positives. 
    
    
    # store image
    cv2.imwrite(place_to_store_results + path.strip('\n'), draw)

# Calculate precision, recall and F1 score 
    

300
../../../03 Data/Simple Dataset/colorIMG_75.json
19
300
../../../03 Data/Simple Dataset/colorIMG_32.json
16
300
../../../03 Data/Simple Dataset/colorIMG_114.json
16
300
../../../03 Data/Simple Dataset/colorIMG_95.json
16
300
../../../03 Data/Simple Dataset/colorIMG_43.json
13
300
../../../03 Data/Simple Dataset/colorIMG_67.json
11
300
../../../03 Data/Simple Dataset/colorIMG_24.json
12
300
../../../03 Data/Simple Dataset/colorIMG_12.json
12
300
../../../03 Data/Simple Dataset/colorIMG_159.json
18
300
../../../03 Data/Simple Dataset/colorIMG_51.json
11
300
../../../03 Data/Simple Dataset/colorIMG_103.json
12
300
../../../03 Data/Simple Dataset/colorIMG_96.json
15
300
../../../03 Data/Simple Dataset/colorIMG_50.json
11
300
../../../03 Data/Simple Dataset/colorIMG_179.json
18
300
../../../03 Data/Simple Dataset/colorIMG_94.json
16
300
../../../03 Data/Simple Dataset/colorIMG_88.json
14
300
../../../03 Data/Simple Dataset/colorIMG_77.json
11
300
../../../03 Data/Simple Dataset/colorIMG